In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into the DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hilo,US,2022-07-04 16:49:17,19.7297,-155.0900,82.27,92,75,6.91
1,1,Hvolsvollur,IS,2022-07-04 16:52:31,63.7533,-20.2243,54.27,58,23,14.00
2,2,Yanam,IN,2022-07-04 16:52:32,16.7333,82.2167,77.92,85,96,4.50
3,3,Rikitea,PF,2022-07-04 16:49:34,-23.1203,-134.9692,73.17,73,16,3.40
4,4,Ketchikan,US,2022-07-04 16:52:33,55.3422,-131.6461,57.67,94,100,5.75


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [10]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False,
                                max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of wind 
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [18]:
# Filter the data set to find cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                         (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hilo,US,2022-07-04 16:49:17,19.7297,-155.0900,82.27,92,75,6.91
2,2,Yanam,IN,2022-07-04 16:52:32,16.7333,82.2167,77.92,85,96,4.50
5,5,Hithadhoo,MV,2022-07-04 16:52:33,-0.6000,73.0833,82.06,73,55,2.26
10,10,Wilmington,US,2022-07-04 16:48:46,34.2257,-77.9447,84.00,82,100,6.91
13,13,Dingle,PH,2022-07-04 16:52:37,10.9995,122.6711,77.94,89,91,1.90
20,20,Trinidad,BO,2022-07-04 16:52:39,-14.8333,-64.9000,89.58,51,80,6.91
23,23,Kavieng,PG,2022-07-04 16:52:41,-2.5744,150.7967,81.97,80,100,14.20
26,26,Isangel,VU,2022-07-04 16:52:42,-19.5500,169.2667,75.43,83,48,12.80
28,28,Puerto Ayora,EC,2022-07-04 16:52:43,-0.7393,-90.3518,75.16,98,87,3.00
32,32,Atuona,PF,2022-07-04 16:52:44,-9.8000,-139.0333,76.77,76,60,23.02


In [19]:
preferred_cities_df.count()

City_ID       195
City          195
Country       195
Date          195
Lat           195
Lng           195
Max Temp      195
Humidity      195
Cloudiness    195
Wind Speed    195
dtype: int64

In [21]:
# Create DataFrame called hotel_df to store hotel names along with city, country, maxtemp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng",]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hilo,US,82.27,19.7297,-155.0900,
2,Yanam,IN,77.92,16.7333,82.2167,
5,Hithadhoo,MV,82.06,-0.6000,73.0833,
10,Wilmington,US,84.00,34.2257,-77.9447,
13,Dingle,PH,77.94,10.9995,122.6711,
20,Trinidad,BO,89.58,-14.8333,-64.9000,
23,Kavieng,PG,81.97,-2.5744,150.7967,
26,Isangel,VU,75.43,-19.5500,169.2667,
28,Puerto Ayora,EC,75.16,-0.7393,-90.3518,
32,Atuona,PF,76.77,-9.8000,-139.0333,


In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make the request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    #Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [25]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hilo,US,82.27,19.7297,-155.0900,Hilo Hawaiian Hotel
2,Yanam,IN,77.92,16.7333,82.2167,The Regency Hotel
5,Hithadhoo,MV,82.06,-0.6000,73.0833,Scoop Guest House
10,Wilmington,US,84.00,34.2257,-77.9447,"Hotel Ballast Wilmington, Tapestry Collection ..."
13,Dingle,PH,77.94,10.9995,122.6711,SEGAYA PROPERTY
20,Trinidad,BO,89.58,-14.8333,-64.9000,Hotel Piesta
23,Kavieng,PG,81.97,-2.5744,150.7967,Nusa Island Retreat
26,Isangel,VU,75.43,-19.5500,169.2667,Tanna Lodge
28,Puerto Ayora,EC,75.16,-0.7393,-90.3518,Finch Bay Galapagos Hotel
32,Atuona,PF,76.77,-9.8000,-139.0333,Villa Enata


In [35]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp/dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [36]:
# Add a heatmap of temperature for vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))